In [31]:
import subprocess
from time import sleep
import json


In [32]:
INPUT_FILE = "ai_test.json"
OUTPUT_CLEAN = "ai_test_clean.json"
OUTPUT_REJECTED = "ai_test_rejected.json"
MODEL = "mistral:instruct"
SCORE_THRESHOLD = 3  # Reject if score <= this

In [33]:
def query_ollama(prompt: str) -> str:
    process = subprocess.run(
        [r"C:\Users\MIRO1\AppData\Local\Programs\Ollama\ollama.exe", "run", MODEL],
        input=prompt.encode(),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        timeout=60,
    )
    return process.stdout.decode()

In [ ]:
def build_prompt(en, fr):
    return f"""
You are a bilingual translation reviewer.

Given the following English sentence and its French translation:

English: "{en}"
French: "{fr}"

Evaluate the quality of the French translation and provide your response in the following JSON format:

{{
  "score": <integer from 1 to 5>,
  "fr": "<corrected French translation or 'REJECTED: <reason>'>"
}}

Scoring Criteria:
- 5: Excellent (accurate and fluent)
- 4: Good (minor grammar/style issues or spelling mistakes)
- 3: Fair (noticeable errors)
- 2: Poor (many errors)
- 1: Bad (completely incorrect)

Only output the JSON object. Do not include any explanations or additional text.
Use ’ instead of ' when writing contractions in french.
"""


In [35]:
def parse_response(response: str):
    try:
        data = json.loads(response)
    except json.JSONDecodeError as e:
        return None, None, f"Invalid JSON: {e}"

    # Validate the presence of 'score' and 'fr' keys
    if 'score' not in data or 'fr' not in data:
        return None, None, "Missing 'score' or 'fr' in response"

    score = data['score']
    fr = data['fr']

    # Ensure 'score' is an integer
    if not isinstance(score, int):
        return None, None, "Score is not an integer"

    # Check if the translation was rejected
    if isinstance(fr, str) and fr.strip().upper().startswith("REJECTED"):
        return score, None, fr.strip()

    return score, fr.strip(), None

In [ ]:
with open(INPUT_FILE, "r", encoding="utf-8") as f_in, \
     open(OUTPUT_CLEAN, "w", encoding="utf-8") as f_clean, \
     open(OUTPUT_REJECTED, "w", encoding="utf-8") as f_reject:

    data = json.load(f_in)
    clean_results = []
    reject_results = []

    for i, row in enumerate(data):
        en = row["en"]
        fr = row["fr"]
        prompt = build_prompt(en, fr)
        try:
            response = query_ollama(prompt)
            score, corrected, reason = parse_response(response)
            if score is None:
                print(f"[ERROR] Line {i+1}: {reason}")
                continue
            if corrected:
                clean_results.append({"en": en, "fr": corrected, "score": score, "original": row["fr"]})
                print(f"[CLEANED] Line {i+1} (Score: {score})")
            else:
                reject_results.append({"en": en, "fr": fr, "score": score, "reason": reason})
                print(f"[REJECTED] Line {i+1} (Score: {score}) - {reason}")
            sleep(1)
        except Exception as e:
            print(f"[ERROR] Line {i+1}: {e}")
            continue

    json.dump(clean_results, f_clean, ensure_ascii=False, indent=2)
    json.dump(reject_results, f_reject, ensure_ascii=False, indent=2)

[CLEANED] Line 1 (Score: 5)
[CLEANED] Line 2 (Score: 5)
[CLEANED] Line 3 (Score: 4)
[CLEANED] Line 4 (Score: 5)
[REJECTED] Line 5 (Score: 1) - REJECTED: This translation does not accurately reflect the original English sentence
